In [ ]:
import pandas as pd
import requests
import json
import time
import os
import base64
import ollama

# Create folder to save images
os.makedirs("downloaded_images", exist_ok=True)

# --- Utility Functions ---


def modify_image_url(url):
    # Logic to Modify Image
    return 


def download_image(image_url, save_dir="downloaded_images"):
    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            file_name = os.path.join(save_dir, image_url.split("/")[-1])
            with open(file_name, "wb") as f:
                f.write(response.content)
            return file_name
        else:
            print(f"Failed to download image: {image_url}")
            return None
    except Exception as e:
        print(f"Error downloading image: {e}")
        return None


def encode_image_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded = base64.b64encode(image_file.read()).decode("utf-8")
    return encoded


def generate_prompt(row):
    # Clear and more focused prompt to reduce hallucination
    return f"""
You are an expert product search assistant.

Given the PRODUCT INFORMATION and PRODUCT IMAGE:
- Generate 6 realistic search keywords or phrases that a user might use to search for this product.
- Focus on attributes like color, size, material, use case, similar items.
- Avoid hallucinating details not seen in text/image.
- Be short, simple, and relevant.
- Also Use image to describe the product and give at least one point regarding color or shape with product category like red color fruit or oval shape fruit.

for example, if its a apple watch, you can say "Grey Color watch, square shape watch, smart watch for fitness, waterproof smart watch, apple watch accessories"

Here are the details:
Product Name: {row['product_name']}
Category: {row['product_category']}
Description: {row['product_description']}

Output only 6 phrases, separated by commas, no extra text.
"""


def generate_search_phrases(prompt, image_path):
    try:
        encoded_image = encode_image_base64(image_path)

        response = ollama.chat(
            model="llava:13b",
            messages=[
                {
                    "role": "system",
                    "content": "You are helping to create realistic and concise search phrases based on product details and an actual product image. Stick closely to visible/textual information. No hallucinations.",
                },
                {
                    "role": "user",
                    "content": prompt,
                    "images": [encoded_image],
                },
            ],
            options={
                "temperature": 0.2,  # Lower temperature => less hallucination
                "top_p": 0.7,  # Slight randomness
                "num_ctx": 4096,  # Enough context window
            },
        )
        return response["message"]["content"]
    except Exception as e:
        print(f"Error calling Ollama API: {e}")
        return None


def process_batch(csv_path, batch_size=10, output_file="fine_tune_data.jsonl"):
    df = pd.read_csv(csv_path)
    examples = []

    for idx in range(0, len(df), batch_size):
        batch = df.iloc[idx : idx + batch_size]
        for _, row in batch.iterrows():
            image_url = modify_image_url(row["product_url"])
            image_path = download_image(image_url)

            if not image_path:
                continue

            prompt = generate_prompt(row)
            search_phrases = generate_search_phrases(prompt, image_path)
            print(f"✅ Generated for {row['product_name']}: {search_phrases}")

            if search_phrases:
                example = {"input": prompt, "output": search_phrases}
                examples.append(example)

            time.sleep(0.5)  # Light delay

    with open(output_file, "w") as f:
        for ex in examples:
            f.write(json.dumps(ex) + "\n")
    print(f"✅ Saved {len(examples)} examples to {output_file}")

In [ ]:
process_batch("products.csv", batch_size=10)